# Navigation

---

*Jun Zhu, zhujun981661@gmail.com, 08.2020*

In this notebook, I present the solution for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).


### 1. Start the Environment

In [ ]:
from unityagents import UnityEnvironment
import numpy as np

!pip install matplotlib

import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F

from dqn_agent import DqnAgent
from utilities import check_environment, play

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [ ]:
env = UnityEnvironment(file_name="Banana_Linux/Banana.x86_64")

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [ ]:
brain_name, state_space, action_space = check_environment(env)
brain_name

### 3. Run the environment with random actions

In [ ]:
# play(env, brain_name)

### 4. Train an agent with deep-Q network (DQN)

The algorithm first proposed in this [nature paper](https://www.nature.com/articles/nature14236) was implemented. In addition, the following enhancements are included:

1. [Dueling network architecture](https://arxiv.org/abs/1511.06581)
2. [Double Q-learning](https://arxiv.org/abs/1509.06461)

In [ ]:
class BananaBrainQNetwork(nn.Module):
    def __init__(self):
        super().__init__()

        # state_space = 37
        # action_space = 4
        self._fc1 = nn.Linear(state_space, 256)
        self._fc2 = nn.Linear(256, 256)
        self._fc3 = nn.Linear(256, 64)
                
        # advantange for each action
        self._fc_adv = nn.Linear(64, action_space)
        # value which is independent of the action
        self._fc_value = nn.Linear(64, 1)

    def forward(self, state):
        x = state
        for fc in (self._fc1, self._fc2, self._fc3):
            x = F.relu(fc(x))
            
        v = self._fc_value(x)
        a = self._fc_adv(x)
        return v + (a - a.mean(1, keepdim=True))

In [ ]:
actions = np.arange(action_space)
target_score = 13

# initialize the model
model = BananaBrainQNetwork()

# initialize the agent
dqn_agent = DqnAgent(model, actions, 
                     replay_memory_size=100000,  # size of the experience replay buffer
                     double_dqn=True  # use double Q-learning
                    )

# train the agent with given hyperparameters (remove ./dqn_checkpoint.pth to train a new model from scratch)
scores = dqn_agent.train(env,
                         n_episodes=2000,
                         epsilon_decay_rate=0.995,  # decay rate of epsilon (1.0 - 0.01)
                         target_network_update_frequency=4,  # frequency of updating target network
                         gamma=0.99,  # discount factor
                         learning_rate=5e-4,  # learning rate
                         batch_size=64,  # mini batch size
                         output_frequency=50,
                         save_frequency=100,
                         target_score=target_score)

Visualize the learning history.

In [ ]:
_, ax = plt.subplots(1, 1, figsize=(12, 6))
x = np.arange(len(scores)) + 1
ax.plot(x, scores, alpha=180, label='score history')
ax.plot(x[99:], np.convolve(scores, np.ones(100) / 100, 'valid'), label='score history (moving averaged)')
ax.plot(x, target_score * np.ones_like(x), '--', label='reward (target)')
ax.set_xlabel("Episode", fontsize=16)
ax.set_ylabel("Score", fontsize=16)
ax.legend()

In [ ]:
dqn_agent.play(env)

In [ ]:
# env.close()

### 5. Further improvement

The following algorithms could further improve the performance of the agent:
    
- [Prioritized experience replay](https://arxiv.org/abs/1511.05952)

  It should be noted that there is performance concern over this algorithm.
 
- [Distributional DQN](https://arxiv.org/abs/1707.06887)
- [Noisy network](https://arxiv.org/abs/1706.10295)